In [142]:
from icecube import icetray, dataio, dataclasses, simclasses, phys_services, recclasses
import os, sys
import numpy as np
import matplotlib as mpl
from matplotlib.colors import LogNorm, Normalize
import matplotlib.pyplot as plt
import timeit as time
import math
from datetime import datetime

In [143]:
# Added by JP
# Define the directory of your files
files_dir = '/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999'
# List the contents of the entire directory
file_list_aux = os.listdir(files_dir)
# Only keep those that are I3 files
file_list = [x for x in file_list_aux if '.i3.bz2' in x]
print('Total files', len(file_list))

Total files 966


In [ ]:
#I would recommend you write a function that takes a frame as input 
#and returns the corrected charge for that event. 
#Inside, the function should pick the muon and loop over all the doms that have seen light, 
#and return the corrected charges of all doms, as an array. So each event gives you an array as the output.

In [144]:
# Now decide how many files to loop over
nfiles = 1 # This can be len(file_list)

In [145]:
def func(frame):
    pulses = frame['SRTInIcePulsesDOMeff'].apply(frame)
    one_dom = pulses.items()
    charge = []
    
    for i in range(len(one_dom)):  
        char = 0
        for pulse in one_dom[i][1]:
            char += pulse.charge        
        if char > 0:
            charge.append(char)
        
    gcd_file = '/cvmfs/icecube.opensciencegrid.org/data/GCD/GeoCalibDetectorStatus_IC86_Merged.i3.gz'
    gfile = dataio.I3File(gcd_file)
    gframe = gfile.pop_frame()
    geometry = gframe['I3Geometry']
    
    mydom = []
    for i in range(len(one_dom)):
        dom = geometry.omgeo[one_dom[i][0]]    
        mydom.append(dom) 
        
    mmctracks = frame['MMCTrackList']
    track = mmctracks[0].particle
    
    distance = []
    for i in range(len(mydom)):
        d = phys_services.I3Calculator.cherenkov_distance(track, mydom[i].position)
        distance.append(d)
    corrected_charge = []
    for i in range(len(distance)):
        att_length = 50. 
        corrected = charge[i]*distance[i]
        corrected_charge.append(corrected)
    
    return corrected_charge

In [150]:
for i in range(nfiles):
    infile = dataio.I3File(os.path.join(files_dir,file_list[i]))
    frame = infile.pop_physics()
    print(len(frame))
    print(len(func(frame)))
    print('Total charge', np.sum(func(frame)) )
    print('Standard deviation', np.std(func(frame)))
    print('Maximum corrected charge observed', np.max(func(frame)) )
    print('Mean corrected charge observed', np.mean(func(frame)) )
    print('Ratio',np.max(func(frame))/np.mean(func(frame)) )

133
51
Total charge 5850.190597420843
Standard deviation 79.81515272200875
Maximum corrected charge observed 420.42546298068703
Mean corrected charge observed 114.70961955727144
Ratio 3.6651282133385497


In [148]:
phys_services.I3Calculator.distance#(between first and last dom?)

<function phys-services.I3Calculator.distance>

In [147]:
for i in range(nfiles):
    with dataio.I3File(os.path.join(files_dir, file_list[i])) as infile:
        for frame in infile:
            if infile.stream.id != 'P': continue
                
            print(len(frame))    
            print(len(func(frame)))
            print('Total charge', np.sum(func(frame)) )
            print('Standard deviation', np.std(func(frame)))
            print('Maximum corrected charge observed', np.max(func(frame)) )
            print('Mean corrected charge observed', np.mean(func(frame)) )
            print('Ratio',np.max(func(frame))/np.mean(func(frame)) )
            

#print(func(frame))

133
51
Total charge 5850.190597420843
Standard deviation 79.81515272200875
Maximum corrected charge observed 420.42546298068703
Mean corrected charge observed 114.70961955727144
Ratio 3.6651282133385497
129
36
Total charge 4899.764818928101
Standard deviation 94.09463146909916
Maximum corrected charge observed 542.2863779954413
Mean corrected charge observed 136.10457830355836
Ratio 3.98433605066512
146
31
Total charge 3468.7989330663177
Standard deviation 94.47281167065323
Maximum corrected charge observed 436.15178967775745
Mean corrected charge observed 111.89673977633282
Ratio 3.8978060535952053
126
20
Total charge 1510.0398739467903
Standard deviation 65.48790599786271
Maximum corrected charge observed 249.18149610412462
Mean corrected charge observed 75.50199369733951
Ratio 3.3003300164894203
131
22
Total charge 2500.242137474859
Standard deviation 85.91812477718334
Maximum corrected charge observed 354.4744078045578
Mean corrected charge observed 113.64736988522087
Ratio 3.11907

KeyboardInterrupt: 

In [109]:

def inf(num):
    infile = dataio.I3File(os.path.join(files_dir,file_list[num]))
    frame = infile.pop_physics()   
    return frame


In [454]:
print(inf(0))


[ I3Frame  (Physics):
  'AnalysisMuons' [Physics] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (174)
  'BackgroundCLSim_intermediatePhotons' [DAQ] ==> I3Map<ModuleKey, I3Vector<I3Photon> > (3167)
  'BackgroundI3MCPESeriesMap_0.990' [DAQ] ==> I3Map<OMKey, vector<I3MCPE> > (90)
  'BackgroundI3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (1786)
  'BackgroundI3MCTree_preMuonProp' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (918)
  'BackgroundMMCTrackList' [DAQ] ==> I3Vector<I3MMCTrack> (304)
  'BeaconLaunches' [DAQ] ==> I3Map<OMKey, vector<I3DOMLaunch> > (46)
  'CalibratedWaveformRange' [DAQ] ==> I3TimeWindow (48)
  'CascadeFilter_13' [Physics] ==> I3PODHolder<bool> (27)
  'CorsikaInteractionHeight' [DAQ] ==> I3PODHolder<double> (36)
  'CorsikaWeightMap' [DAQ] ==> I3Map<string, double> (482)
  'DCAnalysisHits' [Physics] ==> I3PODHolder<double> (36)
  'DCNHits' [Physics] ==> 

In [463]:
pulses = inf(0)['SRTInIcePulsesDOMeff'].apply(inf(0))
one_dom = pulses.items()

In [464]:
charge = []
for i in range(16):  #Sasha: I have 16 here, because it is the first time when we met this "triplet" 
    
    for pulse in one_dom[i][1]:
        char = 0
        char += pulse.charge
        print(char) #Sasha: there are three last values of charge
    charge.append(char) #Sasha: they do not write down in this array, only one of them is written.  
        
print(charge)
#Sasha: but in the next cell I consider them as a sum, I mean I will have not 1.17, 0.925, 1.225 separately, but their sum

0.824999988079071
1.1749999523162842
0.875
0.824999988079071
0.2750000059604645
1.475000023841858
1.1749999523162842
0.824999988079071
1.125
1.5750000476837158
1.4249999523162842
0.32499998807907104
1.0750000476837158
1.475000023841858
0.875
1.1749999523162842
0.925000011920929
1.225000023841858
[0.824999988079071, 1.1749999523162842, 0.875, 0.824999988079071, 0.2750000059604645, 1.475000023841858, 1.1749999523162842, 0.824999988079071, 1.125, 1.5750000476837158, 1.4249999523162842, 0.32499998807907104, 1.0750000476837158, 1.475000023841858, 0.875, 1.225000023841858]


In [487]:
charge = []
for i in range(len(one_dom)):  
    char = 0
    for pulse in one_dom[i][1]:
        char += pulse.charge
        
        
    charge.append(char)
        
print(charge)

[0.824999988079071, 1.1749999523162842, 0.875, 0.824999988079071, 0.2750000059604645, 1.475000023841858, 1.1749999523162842, 0.824999988079071, 1.125, 1.5750000476837158, 1.4249999523162842, 0.32499998807907104, 1.0750000476837158, 1.475000023841858, 0.875, 3.324999988079071, 0.625, 3.3000000715255737, 1.125, 0.7749999761581421, 1.375, 0.925000011920929, 1.0750000476837158, 0.675000011920929, 1.375, 1.3250000476837158, 1.0750000476837158, 1.3499999642372131, 1.375, 0.875, 2.649999976158142, 0.7250000238418579, 0.4749999940395355, 1.4249999523162842, 0.7749999761581421, 1.024999976158142, 1.4249999523162842, 0.7250000238418579, 0.22499999403953552, 1.125, 1.1749999523162842, 1.274999976158142, 0.875, 0.925000011920929, 0.42500001192092896, 0.875, 2.574999988079071, 0.875, 2.900000035762787, 3.774999976158142, 3.475000113248825]


In [466]:
gcd_file = '/cvmfs/icecube.opensciencegrid.org/data/GCD/GeoCalibDetectorStatus_IC86_Merged.i3.gz'

gfile = dataio.I3File(gcd_file)
gframe = gfile.pop_frame()
print(gframe)
geometry = gframe['I3Geometry']

[ I3Frame  (Geometry):
  'I3Geometry' [Geometry] ==> I3Geometry (641528)
]



In [467]:
mydom = []
for i in range(len(one_dom)):
    dom = geometry.omgeo[one_dom[i][0]]    
    mydom.append(dom)  
    
print(len(mydom))
#print(mydom[0])


51


In [478]:
mmctracks = inf(0)['MMCTrackList']
track = mmctracks[0].particle
print(track)


[ I3Particle MajorID : 13469127016857373990
             MinorID : 846
              Zenith : 0.400847
             Azimuth : 1.80333
                   X : -227.484
                   Y : 1017.22
                   Z : 1949.99
                Time : 3536.68
              Energy : 1054.01
               Speed : 0.299792
              Length : 3125.59
                Type : MuPlus
        PDG encoding : -13
               Shape : StartingTrack
              Status : NotSet
            Location : InIce
]


In [482]:
distance = []
for i in range(len(mydom)):
    d = phys_services.I3Calculator.cherenkov_distance(track, mydom[i].position)
    distance.append(d)
print(len(distance))

51


In [488]:
corrected_charge = []
for i in range(len(distance)):
# This is for adding also the attenuation (absorption + scattering)
    att_length = 50. # Typical attenuation length in icecube
    corrected = charge[i]*distance[i]#*np.exp(distance/att_length)
    corrected_charge.append(corrected)

In [490]:
print(len(corrected_charge))
print(corrected_charge)

51
[222.0763055078901, 167.65234550125294, 90.87879437146664, 70.22624541320249, 32.3280058807941, 87.84635247531457, 58.48567357188964, 43.853323474581885, 199.65093749494238, 205.83572456677518, 101.78875708003031, 20.087685112520425, 56.326474720920636, 63.917247745306305, 30.530444785473144, 91.92426364795048, 14.230792672602277, 84.02980064662638, 145.34649795757636, 111.39816999261477, 420.42546298068703, 200.25207647815236, 175.6844459128978, 94.48273255331011, 161.179778497579, 140.79109625569555, 110.38414462387806, 80.54971137983249, 79.88749828800925, 48.8141967257675, 155.96482979729348, 45.35664547689054, 75.50349198575303, 351.0134498294823, 153.79620421796307, 200.1332633587479, 269.3021013764831, 134.78485505776138, 48.89932423766513, 115.99762021723895, 98.24075255388614, 101.74091060807486, 63.26580603753744, 63.48418668450139, 23.245831617524548, 39.831076130185615, 110.32993059738524, 35.376758412388426, 106.06636885481633, 128.34913355697762, 118.64307049674646]


In [36]:
# Try to put it inside a function so that you can pass the puse series to the function, 
# and obtain a list of corrected charges
# Consider looking only at DOMs "nearby" (within a maximum distance of 150-200m, no further)

In [37]:
phys_services.I3Calculator.closest_approach_distance

<function phys-services.I3Calculator.closest_approach_distance>

In [147]:
nfiles = 2

char = []

for i in range(nfiles):
    with dataio.I3File(os.path.join(files_dir, file_list[i])) as infile:
        for frame in infile:  
            frame = infile.pop_physics()             
            fr.append(frame)
            pulses = frame['SRTInIcePulsesDOMeff'].apply(frame)
        
print(len(pulses))
#print(pulses)
zero = []
one = []


for i in range(len(pulses)):
    one_dom = pulses.popitem()
   
    charge = 0
    for pulse in one_dom[1]:
        charge += pulse.charge
    #print(charge)
    char.append(charge)
    zero.append(one_dom[0])
    one.append(one_dom[1])
    #print(one_dom[0])
    
print(len(char))
#print(char)
#print(zero)

88
88


In [148]:
gcd_file = '/cvmfs/icecube.opensciencegrid.org/data/GCD/GeoCalibDetectorStatus_IC86_Merged.i3.gz'

gfile = dataio.I3File(gcd_file)

dom=[]
tr=[]
dist = []


for i in range(nfiles):
    with dataio.I3File(os.path.join(files_dir, file_list[i])) as infile:
        for frame in infile:  
            frame = infile.pop_physics()             
            fr.append(frame)
        
            mmctracks = frame['MMCTrackList']
            track = mmctracks[0].particle
            tr.append(track)
        
        gframe = gfile.pop_frame()    
        geometry = gframe['I3Geometry']

for j in range(len(zero)):
    mydom = geometry.omgeo[zero[j]]
    dom.append(mydom)
           
    distance = phys_services.I3Calculator.cherenkov_distance(track, mydom.position)
    dist.append(distance)
        
print(len(dist))    
print(len(tr))
print(len(dom))

88
2588
88


In [173]:
def cor_char(char, dist):

  corrected_charge = char * dist
  return corrected_charge

cor_char(5,10)

50

50

In [58]:
list_of_char = []

for charge in range(2):
    for distance in range(2):
        list_of_char.append(cor_char(charge,distance))

In [59]:
print(list_of_char)

[0, 0, 0, 1]
